#  Sentinel-1 Level-1B IFREMER variables explanation

Although some variables need extra explanation about why there are present, how they are computed and how to properly use them.

A dedicated Python library is available to use IFREMER Sentinel-1 Level-1B products: https://slcl1butils.readthedocs.io

In [ ]:
from xsarslc.get_test_files import get_test_file
import xarray as xr
# https://cyclobs.ifremer.fr/static/sarwing_datarmor/xsardata/S1B_IW_XSP__1SDV_20200925T173737_20200925T173804_023535_02CB5D_E473.SAFE/s1b-iw1-slc-vv-20200925t173739-20200925t173804-023535-02cb5d-004_L1B_xspec_IFR_1.4k.nc.zip
#localpath = get_test_file('S1B_IW_XSP__1SDV_20200925T173737_20200925T173804_023535_02CB5D_E473.SAFE/s1b-iw1-slc-vv-20200925t173739-20200925t173804-023535-02cb5d-004_L1B_xspec_IFR_1.4k.nc')
localpath = get_test_file('s1b-iw1-slc-vv-20200925t173739-20200925t173804-023535-02cb5d-004_L1B_xspec_IFR_1.4k.nc')
localpath

In [ ]:
import os
#localpath_tweaked = os.path.join(os.path.dirname(os.path.dirname(localpath)),os.path.basename(localpath))
#print(localpath_tweaked,os.path.exists(localpath_tweaked))
print(os.path.exists(localpath))
ds = xr.open_dataset(localpath,group='intraburst')
ds

## wavenumbers in range axis: "k_rg"

"k_rg" variable slightly depends on each tile because the pixel spacing is not the same in the IW sub-swath. 

We made the choice to have a constant dk (in range and azimuth) for all the tiles/cross-spectrum, and we achieved a 10e-3 consistency as a maximum point-to-point difference wavenumbers . 

This strategy allows to compare a spectra to another.

In [ ]:
k_rg1 = ds['k_rg'].isel(burst=2,tile_sample=1)
k_rg2 = ds['k_rg'].isel(burst=2,tile_sample=3)
k_rg1

In [ ]:
from matplotlib import pyplot as plt
k_rg1.plot(label="burst=2,tile_sample=1",linestyle='--',lw=5)
k_rg2.plot(label="burst=2,tile_sample=2")
plt.legend()

In [ ]:
(k_rg1-k_rg2).plot()
plt.title('difference between 2 different k_rg')

The difference between the 2 vectors of wave numbers is super small but for users who might want to stack the different cross spectrum, a first step would be to assign a single k_rg to all the cross spectrum.

## SAR image complexe cartesian cross spectrum

The "xpsectra_{0-1-2}tau_*" is split in real and imaginary part because netCDF4 file format does not allow to store complex values.


Also only half of the spectrum along the range wavenumbers is stored in the netCDF files to save space.


In [ ]:
# recompose complexe SAr image cross spectrum from real and imaginary part
for tautau in range(3):
    ds['xspectra_%stau'%tautau] = ds['xspectra_%stau_Re'%tautau] + 1j*ds['xspectra_%stau_Im'%tautau]
    #ds = ds.drop(['xspectra_%stau_Re'%tautau,'xspectra_%stau_Im'%tautau])
one_cartesian_xsp = ds['xspectra_2tau'].isel({'burst':6,'tile_line':0,'tile_sample':2,'2tau':0})
one_cartesian_xsp = one_cartesian_xsp.assign_coords({'freq_line':one_cartesian_xsp.k_az,'freq_sample':one_cartesian_xsp.k_rg})
one_cartesian_xsp

In [ ]:
from matplotlib import colors as mcolors
cmap = mcolors.LinearSegmentedColormap.from_list("", ["white","violet","mediumpurple","cyan","springgreen","yellow","red"])

abs(one_cartesian_xsp.real).plot(cmap=cmap)
plt.grid(True)

## landmask

The "land_flag" annotated in the sample of L1B product comes from `cartopy` 10 m resolution polygons. 

cartopy/shapefiles/natural_earth/physical/: https://www.naturalearthdata.com/

In [ ]:
ds['land_flag'].plot()

## variance of cross spectrum

"var_xspectra_{0-1-2}tau" is the variance of the different periodograms while "xpsectra_{0-1-2}tau" is the mean of the periodograms computed within a given tile.

These variables **var_xspectra_{0-1-2}tau** are defined on the same grid than the cross spectrum **xpsectra_{0-1-2}tau**. They are also splitted in Real part and Imaginary part in the netCDF files, and they are also stored on the half part of the wavenumer along range axis since they are symmetric.


In [ ]:
ds['var_xspectra_2tau'].isel(tile_line=0,tile_sample=0,burst=3).squeeze().plot(cmap=cmap)